# 笔记

## 概括(AI 生成+自己完善)

本教程是 PyTorch 快速入门系列的核心部分，它将前面章节介绍的概念（数据加载、模型构建、自动求导）整合起来，完整地演示了模型参数的优化过程。教程首先强调了**超参数**（如学习率、批量大小、周期数）在控制模型训练过程中的重要性。接着，它详细解释了**优化循环**的两个主要组成部分：**训练循环**和**验证/测试循环**。在训练循环中，教程重点介绍了**损失函数**（以 `nn.CrossEntropyLoss` 为例）如何衡量模型预测与真实标签的差异，以及**优化器**（以 `torch.optim.SGD` 为例）如何根据计算出的梯度来调整模型参数。优化步骤（`optimizer.zero_grad()`、`loss.backward()`、`optimizer.step()`）被清晰地阐述。最后，教程提供了 `train_loop` 和 `test_loop` 函数的完整实现，并通过在 FashionMNIST 数据集上运行多个周期来展示模型性能的逐步提升。

## 关键函数、语法(自己总结+AI 优化)

- **Hyperparameters (超参数)**:
    -   是在训练开始前设置的参数，用于控制学习过程本身。
    -   常见的超参数包括：
        -   `learning_rate` (学习率): 控制每次参数更新的幅度。
        -   `batch_size` (批量大小): 每次通过网络传播并更新参数的样本数量。
        -   `epochs` (周期数): 整个训练数据集迭代通过模型的次数。
    -   `batch_size` 和 `epochs` 有什么区别？
        -   `batch_size` 是**每次**通过网络传播并更新参数的**样本数量**。
        -   `epochs` 是**整个**训练数据集迭代通过模型的**次数**。

- `tensor.type(new_dtype)`:
    -   Tensor 的方法，用于将其数据类型转换为指定的 `new_dtype`。
    -   例如，`(pred.argmax(1) == y).type(torch.float)` 将布尔类型的比较结果张量 (其中 `True` 表示预测正确，`False` 表示错误) 转换为浮点型张量 (`True` 变为 `1.0`，`False` 变为 `0.0`)。
    -   在计算分类准确率等指标时非常有用，因为转换后的浮点型张量可以直接求和 (`.sum()`) 来统计正确预测的数量。

## 提问与解答(自己提问+AI 回答)

- **问题1 (基于对 "Hyperparameters" 的定义):**
  教程定义了学习率 (learning rate)、批量大小 (batch size) 和周期数 (epochs) 作为关键超参数。
    1.  这三个超参数在模型优化过程中是如何相互作用的？例如，如果显著减小 `batch_size`，需要如何调整 `learning_rate` 或 `epochs` 来达到类似的训练效果，其背后的原因是什么？
    2.  “Smaller values [for learning rate] yield slow learning speed, while large values may result in unpredictable behavior during training.” 除了学习速度和不可预测行为，过大或过小的学习率还会分别带来其他哪些问题？

  - **解答:**
    1.  **超参数的相互作用:**
        -   **`batch_size` 与 `epochs`**: 总的训练迭代次数 (即参数更新的次数) 可以粗略地认为是 `(数据集总样本数 / batch_size) * epochs`。如果减小 `batch_size`，那么在一个 epoch 内参数更新的次数会增加。为了让模型看到相同数量的总样本，可能不需要改变 `epochs`；但由于每次更新的梯度是基于更少样本计算的，可能噪声更大，训练过程可能更不稳定，可能需要更多的 `epochs` 才能收敛，或者需要调整学习率。
        -   **`batch_size` 与 `learning_rate`**:
            -   当 `batch_size` 减小时，每个批次计算的梯度估计的方差通常会增大（因为基于更少的样本）。如果保持学习率不变，这种噪声较大的梯度更新可能会导致训练震荡。因此，一种常见的启发式做法是，当 `batch_size` 减小 (例如减半) 时，也相应地减小学习率 (例如，除以 $\sqrt{2}$ 或减小一个较小的固定因子)，以维持训练的稳定性。
            -   反之，当 `batch_size` 增大时，梯度估计更准确，方差更小，可以尝试使用稍大一些的学习率来加速收敛（例如，线性缩放规则：如果 `batch_size` 乘以 k，学习率也乘以 k，但这有其适用范围和限制）。
        -   **`learning_rate` 与 `epochs`**: 如果学习率设置得非常小，模型参数更新缓慢，可能需要更多的 `epochs` 才能达到较好的收敛状态。如果学习率较大，模型可能较快地接近最优解，但也可能在最优点附近震荡，此时过多的 `epochs` 可能不会带来进一步的显著提升，甚至可能导致过拟合。学习率调度器 (Learning Rate Schedulers) 常用来在训练过程中动态调整学习率，例如开始时用较大学习率，然后逐渐减小。

    2.  **学习率过大/过小的其他问题:**
        -   **学习率过大**:
            -   **发散 (Divergence)**: 损失值非但没有减小，反而持续增大，导致模型完全无法学习。
            -   **跳过最优解**: 即使没有完全发散，也可能在损失函数的最小值附近剧烈震荡，无法稳定在最优或接近最优的点。
            -   **数值不稳定**: 极端的学习率可能导致权重值变得非常大或非常小，引发数值溢出 (NaN 或 inf)。
        -   **学习率过小**:
            -   **收敛极其缓慢**: 需要非常长的时间和大量的计算资源才能达到可接受的性能。
            -   **陷入次优的局部最小值**: 由于更新步长太小，优化器可能缺乏足够的“动能”跳出较差的局部最小值或鞍点。
            -   **对初始化更敏感**: 如果初始化的参数位置不好，学习率过小可能使得模型难以摆脱这个初始状态。

    参考资料:
    -   "Practical recommendations for gradient-based training of deep architectures" by Yoshua Bengio (Section 3.1 discusses batch size and learning rates). [https://arxiv.org/abs/1206.5533](https://arxiv.org/abs/1206.5533)
    -   Deep Learning Specialization (Coursera, Andrew Ng) - Course on Hyperparameter Tuning.

- **问题2 (关于 "Loss Function" 和 "Optimizer"):**
  教程中使用了 `nn.CrossEntropyLoss` 作为损失函数，`torch.optim.SGD` 作为优化器。
    1.  教程提到 `nn.CrossEntropyLoss` 结合了 `nn.LogSoftmax` 和 `nn.NLLLoss`。这是否意味着如果使用了 `nn.CrossEntropyLoss`，则模型的原始输出 (logits) **不应该**再经过一个外部的 Softmax 层，是这样的吗？如果在模型输出后加了 Softmax 再传入 `nn.CrossEntropyLoss`，可能会出现什么问题？
    2.  SGD 是一种基础的优化算法。教程中提到了 ADAM 和 RMSProp 作为其他可用的优化器。这些自适应学习率优化器（如 ADAM）与 SGD 相比，在哪些方面更有优势？是否存在 SGD 仍然是更好的选择的情况？

  - **解答:**
    1.  **`nn.CrossEntropyLoss` 与 Softmax:**
        -   **原因**: `nn.CrossEntropyLoss` 在其内部实现中已经包含了计算 Softmax (更准确地说是 `LogSoftmax`，即 $\log(\text{Softmax}(x))$) 的步骤。它期望的输入是网络直接输出的原始、未归一化的分数（logits）。
        -   **错误添加外部 Softmax 的问题**:
            -   **数学错误**: 如果你先对 logits 应用 Softmax (得到概率 $p_i$)，然后再将这些概率传入 `nn.CrossEntropyLoss`，那么损失函数内部的 `LogSoftmax` 会再次尝试处理这些已经处于 $[0,1]$ 范围内的概率值。这会导致数学上的计算与期望的交叉熵损失不同，很可能使得损失函数的表现不佳，模型难以正确学习。
            -   **数值稳定性**: 将 `LogSoftmax` 和 `NLLLoss` 结合在 `CrossEntropyLoss` 中的一个主要原因是数值稳定性。分开计算可能会导致数值下溢 (log(0)) 或精度问题。如果外部应用了 Softmax，就失去了这种数值优化的好处。
            -   **梯度问题**: 错误的计算流程可能导致梯度不正确或表现怪异。

    2.  **ADAM/RMSProp vs. SGD:**
        -   **自适应优化器的优势 (如 ADAM, RMSProp):**
            -   **自适应学习率**: 它们为每个参数维护一个单独的学习率，并根据该参数历史梯度的大小（通常是一阶矩和二阶矩的估计）来动态调整学习率。这使得它们对于不同参数的缩放和稀疏性不那么敏感。
            -   **通常更快的收敛**: 在许多实际任务中，尤其是在具有复杂损失曲面或稀疏梯度的问题上，自适应优化器往往能比标准 SGD（无动量）更快地收敛。
            -   **对超参数选择的鲁棒性**: 通常对初始学习率的选择不如 SGD 敏感，更容易找到一组能工作得不错的超参数。
        -   **SGD 的潜在优势或适用场景:**
            -   **更好的泛化能力 (有时)**: 一些研究和实践表明，尽管自适应优化器可能在训练集上收敛更快，但 SGD (尤其是带动量的 SGD) 有时能找到泛化能力更好（即在未见过的数据上表现更好）的解。这被认为可能与自适应方法倾向于收敛到更尖锐的最小值有关，而 SGD 可能找到更平坦的最小值。
            -   **简单且易于理解/调整**: SGD 的机制相对简单。对于某些问题，精心调整的 SGD (带动量和学习率调度) 仍然可以达到最佳性能。
            -   **内存占用**: 一些自适应优化器需要为每个参数存储额外状态 (如梯度的一阶和二阶矩的移动平均值)，这会增加内存消耗，尽管对于大多数现代GPU来说这通常不是主要瓶颈。
            -   **对于某些特定架构或问题**: 在一些非常特定的情况下，SGD 可能由于其特定的优化路径行为而被证明更有效。

        总的来说，ADAM 因其良好的综合性能和易用性，常作为许多任务的默认首选。但 SGD 及其变种（如 SGD with Momentum）仍然是非常强大且广泛使用的优化器，特别是在追求极致泛化性能或对优化过程有更细致控制需求的场景中。

    参考资料:
    -   PyTorch `nn.CrossEntropyLoss` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
    -   "An overview of gradient descent optimization algorithms" by Sebastian Ruder (提供了对各种优化器的优秀比较): [https://ruder.io/optimizing-gradient-descent/](https://ruder.io/optimizing-gradient-descent/)

- **问题3 (关于 `train_loop` 和 `test_loop` 中的 `model.train()` 和 `model.eval()`):**
  教程中的 `train_loop` 函数以 `model.train()` 开始，而 `test_loop` 函数以 `model.eval()` 开始，并专门强调这对于批标准化 (batch normalization) 和 dropout 层很重要。
    1.  请详细解释批标准化层 (如 `nn.BatchNorm2d`) 和 Dropout 层 (如 `nn.Dropout`) 在 `model.train()` 模式和 `model.eval()` 模式下的行为有何不同？
    2.  为什么这种不同对于模型的训练和评估十分重要？如果忘记调用 `model.train()` 或 `model.eval()`，可能会产生什么影响？

  - **解答:**
    1.  **`nn.BatchNorm` 和 `nn.Dropout` 在不同模式下的行为:**
        -   **批标准化 (Batch Normalization):**
            -   **`model.train()` 模式**:
                -   BatchNorm 层会计算当前输入批次数据的均值和方差。
                -   使用这些批次统计量来归一化当前批次的激活值。
                -   同时，它还会更新一个运行中的 (running) 均值和方差的估计 (通常通过指数移动平均)，这些运行统计量将在评估阶段使用。
                -   可学习的仿射参数 $\gamma$ 和 $\beta$ 会被更新。
            -   **`model.eval()` 模式**:
                -   BatchNorm 层**不再**计算当前输入批次的均值和方差，也**不更新**运行统计量。
                -   它使用在整个训练过程中学习到的固定的运行均值和方差来归一化输入。
                -   可学习的仿射参数 $\gamma$ 和 $\beta$ 是固定的（不更新）。
        -   **Dropout 层:**
            -   **`model.train()` 模式**:
                -   Dropout 层会以指定的概率 $p$ 随机地将输入张量中的一部分元素置为零（即“丢弃”它们）。
                -   为了保持激活值的期望总和不变，未被置零的元素通常会被放大 $1/(1-p)$ 倍 (inverted dropout)。
            -   **`model.eval()` 模式**:
                -   Dropout 层**不起作用**，即它不会随机丢弃任何激活值。它相当于一个恒等变换，所有输入都原样通过。

    2.  **行为差异的重要性及忘记 `model.train()` 或 `model.eval()` 的影响:**
        -   **重要性:**
            -   **对于 BatchNorm**:
                -   在训练时，使用批次统计量可以帮助稳定训练过程，减少内部协变量偏移。更新运行统计量是为了在评估时有一个对整个数据集特征分布的稳定估计。
                -   在评估时，必须使用固定的运行统计量。如果继续使用当前测试批次的统计量，那么模型的输出会依赖于该测试批次中其他样本，这会使评估结果不稳定且不具代表性（例如，单个样本的预测会因其所在的测试批次不同而改变）。
            -   **对于 Dropout**:
                -   在训练时，Dropout 是一种正则化技术，通过引入噪声和阻止神经元之间的协同适应来防止过拟合。
                -   在评估时，我们希望模型使用其所有学到的能力来进行确定性的预测，而不是随机丢弃神经元。因此，Dropout 必须被禁用。
        -   **忘记 `model.train()` 在 `train_loop` 中的影响:**
             -   **BatchNorm 的影响**
                 -   **运行统计量不更新**: BatchNorm 层会使用其固定的（可能是训练开始时的初始值，或者加载模型时的值）运行均值和运行方差来对当前训练批次进行归一化，并且**不会**根据当前批次的数据来更新这些运行统计量。这意味着，在整个训练过程中，用于未来评估的运行统计量没有得到正确的学习和积累。当训练完成后，模型在评估模式下会使用这些未被正确训练的运行统计量，导致模型性能可能远差于预期。
                 -   **训练稳定性可能下降或学习效果变差**: 训练时，BatchNorm 的一个重要作用是利用批次统计量来稳定学习过程。如果处于 `eval` 模式，它使用的是固定的全局统计量，这可能与当前批次的实际分布有较大差异，可能导致训练不稳定或模型学习效果不佳。
             -   **Dropout 的影响**
                 -   **Dropout 失效**: Dropout 层将不会执行任何操作，即不会随机丢弃神经元。
                 -   **缺乏正则化**: 模型将失去 Dropout 提供的正则化效果，这会增加模型在训练集上过拟合的风险，从而降低其在未见过数据（如验证集或测试集）上的泛化能力。
             -   一般而言，模型会默认处于 `train` 模式，因此忘记调用 `model.train()` 通常不会导致明显的问题。但是当你在训练过程中需要使用 `eval` 模式时，忘记调用 `model.train()` 可能会导致模型在后续的训练中使用错误的模式，从而影响训练的正确性和效果。
        -   **忘记 `model.eval()` 在 `test_loop` 中的影响:**
            -   **BatchNorm 的影响**: 如果模型包含 BatchNorm 层，并且在测试时忘记调用 `model.eval()`，那么 BatchNorm 层会继续使用当前测试批次的均值和方差进行归一化，并可能（错误地）更新其运行统计量。这会导致：
                -   **不稳定的评估结果**: 测试准确率和损失可能会因测试批次的不同而有较大波动。
                -   **性能通常会更差**: 因为批次统计量可能与全局统计量有很大差异，导致归一化不当。
                -   运行统计量被污染，如果后续还需要使用模型进行推理。
            -   **Dropout 的影响**: 如果模型包含 Dropout 层，并且在测试时忘记调用 `model.eval()`，那么 Dropout 层会继续随机丢弃激活。这会导致：
                -   **性能显著下降**: 模型没有使用其全部学到的权重，而是随机地“关闭”了一部分网络，导致预测能力减弱。
                -   **不确定性的预测**: 对于同一个输入样本，多次运行测试可能会得到不同的预测结果（如果随机种子不同或实现依赖于随机性）。

        因此，正确设置 `model.train()` 和 `model.eval()` 是确保模型训练得当、评估公平且结果可复现的关键步骤。

    参考资料:
    -   PyTorch `nn.BatchNorm2d` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)
    -   PyTorch `nn.Dropout` 文档: [https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)


[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\| **Optimization** \|\| [Save &
Load Model](saveloadrun_tutorial.html)

Optimizing Model Parameters
===========================

Now that we have a model and data it\'s time to train, validate and test
our model by optimizing its parameters on our data. Training a model is
an iterative process; in each iteration the model makes a guess about
the output, calculates the error in its guess (*loss*), collects the
derivatives of the error with respect to its parameters (as we saw in
the [previous section](autograd_tutorial.html)), and **optimizes** these
parameters using gradient descent. For a more detailed walkthrough of
this process, check out this video on [backpropagation from
3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

Prerequisite Code
-----------------

We load the code from the previous sections on [Datasets &
DataLoaders](data_tutorial.html) and [Build
Model](buildmodel_tutorial.html).


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Hyperparameters
===============

Hyperparameters are adjustable parameters that let you control the model
optimization process. Different hyperparameter values can impact model
training and convergence rates ([read
more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)
about hyperparameter tuning)

We define the following hyperparameters for training:

:   -   **Number of Epochs** - the number times to iterate over the
        dataset
    -   **Batch Size** - the number of data samples propagated through
        the network before the parameters are updated
    -   **Learning Rate** - how much to update models parameters at each
        batch/epoch. Smaller values yield slow learning speed, while
        large values may result in unpredictable behavior during
        training.


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Optimization Loop
=================

Once we set our hyperparameters, we can then train and optimize our
model with an optimization loop. Each iteration of the optimization loop
is called an **epoch**.

Each epoch consists of two main parts:

:   -   **The Train Loop** - iterate over the training dataset and try
        to converge to optimal parameters.
    -   **The Validation/Test Loop** - iterate over the test dataset to
        check if model performance is improving.

Let\'s briefly familiarize ourselves with some of the concepts used in
the training loop. Jump ahead to see the
`full-impl-label`{.interpreted-text role="ref"} of the optimization
loop.

Loss Function
-------------

When presented with some training data, our untrained network is likely
not to give the correct answer. **Loss function** measures the degree of
dissimilarity of obtained result to the target value, and it is the loss
function that we want to minimize during training. To calculate the loss
we make a prediction using the inputs of our given data sample and
compare it against the true data label value.

Common loss functions include
[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error) for regression tasks, and
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood) for classification.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model\'s output logits to `nn.CrossEntropyLoss`, which will
normalize the logits and compute the prediction error.


In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Optimizer
=========

Optimization is the process of adjusting model parameters to reduce
model error in each training step. **Optimization algorithms** define
how this process is performed (in this example we use Stochastic
Gradient Descent). All optimization logic is encapsulated in the
`optimizer` object. Here, we use the SGD optimizer; additionally, there
are many [different
optimizers](https://pytorch.org/docs/stable/optim.html) available in
PyTorch such as ADAM and RMSProp, that work better for different kinds
of models and data.

We initialize the optimizer by registering the model\'s parameters that
need to be trained, and passing in the learning rate hyperparameter.


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:

:   -   Call `optimizer.zero_grad()` to reset the gradients of model
        parameters. Gradients by default add up; to prevent
        double-counting, we explicitly zero them at each iteration.
    -   Backpropagate the prediction loss with a call to
        `loss.backward()`. PyTorch deposits the gradients of the loss
        w.r.t. each parameter.
    -   Once we have our gradients, we call `optimizer.step()` to adjust
        the parameters by the gradients collected in the backward pass.


Full Implementation {#full-impl-label}
===================

We define `train_loop` that loops over our optimization code, and
`test_loop` that evaluates the model\'s performance against our test
data.


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to
`train_loop` and `test_loop`. Feel free to increase the number of epochs
to track the model\'s improving performance.


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.316605  [   64/60000]
loss: 2.300218  [ 6464/60000]
loss: 2.284449  [12864/60000]
loss: 2.274445  [19264/60000]
loss: 2.255760  [25664/60000]
loss: 2.235267  [32064/60000]
loss: 2.235708  [38464/60000]
loss: 2.208260  [44864/60000]
loss: 2.212806  [51264/60000]
loss: 2.165894  [57664/60000]
Test Error: 
 Accuracy: 43.4%, Avg loss: 2.166979 

Epoch 2
-------------------------------
loss: 2.185601  [   64/60000]
loss: 2.170972  [ 6464/60000]
loss: 2.116535  [12864/60000]
loss: 2.122788  [19264/60000]
loss: 2.087616  [25664/60000]
loss: 2.030255  [32064/60000]
loss: 2.049578  [38464/60000]
loss: 1.980528  [44864/60000]
loss: 1.989454  [51264/60000]
loss: 1.900577  [57664/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.908406 

Epoch 3
-------------------------------
loss: 1.952561  [   64/60000]
loss: 1.917697  [ 6464/60000]
loss: 1.802959  [12864/60000]
loss: 1.826016  [19264/60000]
loss: 1.745511  [25664/60000]
loss: 1.693929  [32064/600

Further Reading
===============

-   [Loss
    Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
-   [torch.optim](https://pytorch.org/docs/stable/optim.html)
-   [Warmstart Training a
    Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
